# Introduction

So far, we've delved into the foundational concepts and practical implementations of clustering, classification, and regression. In this notebook, we will explore how these machine learning techniques can be applied to solve complex problems in the field of material science.

Understanding the behavior of materials at an atomic level is crucial for the development of new materials with enhanced properties. Traditionally, these insights are obtained through experiments and simulations, with classical molecular dynamics (MD) being a prominent simulation technique. In classical molecular dynamics, we simulate the interactions between atoms over time, providing detailed information about the system's evolution. However, as we will see bellow, analyzing and interpreting this data can be challenging due to its complexity and high dimensionality. This is where machine learning comes into play, offering powerful tools to extract meaningful patterns and make predictions.

Throughout this notebook, we will import data generated from classical molecular dynamics simulations and demonstrate how to preprocess, analyze, and visualize this data using machine learning techniques.

In [ ]:
# Collectiong all the data and packages we will need:
!rm -r /content/Notebooks
!git clone https://github.com/jpalastus/Notebooks.git
!pip install ase dscribe

-------------------------------------------------------------------------
# Clustering Globals Structure Descriptors from Copper Melting MD (Easy case!)

Here, we provided you some data that come from a LAMMPS molecular dynamicsof a 1000 Cu atoms FCC supercell from 1000-2000K, at constant pressure of 1atm, using the Morse potential. To make the things easy here, we already have a descriptor of those structures generated for you (with the script provided in the github). Our job here is to try to interpret this data and separate the liquid and solid structures one from the other. To do so, we will use some clustering in the [MBTR descriptor](https://singroup.github.io/dscribe/latest/tutorials/descriptors/mbtr.html) with $k=2$ (equivalent to well known $g(r)$).

In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.manifold import TSNE, locally_linear_embedding
from sklearn.decomposition import PCA
import numpy as np

X = np.load('/content/Notebooks/Oleron Solidification School 2024/data/MD_melting/mbtr_copper.npy')
X=X[1:] #We are removing the initial frame (perfect FCC).

In [ ]:
from matplotlib import cm,colormaps

N=len(X)
print(N)
cmap = colormaps['jet']
colours=[cmap(float(i)/float(N-1)) for i in range(N)]

xmin=1.5; xmax=5.0

plt.figure(figsize=(14, 6))
plt.xlabel("Bound Length ($\AA$)")
plt.ylabel("Cluster-averaged MBTR for k=2 (Radial Distribution)")
for i in range(1,len(X)):
  xx=X[i]#+0.001*i    <--------- Try adding this!
  N=len(xx)
  xx_plot=(np.array(range(N))/float(N))*(xmax-xmin)+xmin
  plt.plot(xx_plot, xx,'-', linewidth=1, c = colours[i])


plt.show()

In [ ]:
from sklearn.cluster import KMeans, DBSCAN
import matplotlib
cm_preference='bwr'

# We are using dbscan!  eps=0.2 and min_sample=10 are `fine tunned` to this aplication.
# You can play with then to see what happens.
model = DBSCAN(eps=0.20,
               min_samples=10,
               metric='l1') # L1 metric here acts as an integral of the absolut difference

model.fit(X)

y_pred = model.labels_
nb_c=max(y_pred)+1

print(nb_c)

cmap = matplotlib.cm.get_cmap(cm_preference)
colours=[cmap(float(i)/float(nb_c-1)) for i in range(nb_c)]

xmin=1.5; xmax=5.0

plt.figure(figsize=(14, 6))
plt.xlabel("Bound Length ($\AA$)")
plt.ylabel("Cluster-averaged MBTR for k=2 (Radial Distribution)")
for i in range(nb_c):
  aux=X[y_pred==i]
  xx=np.mean(np.array(aux), axis=0)
  N=len(xx)
  xx_plot=(np.array(range(N))/float(N))*(xmax-xmin)+xmin
  plt.plot(xx_plot, xx,'-', linewidth=5, c = colours[i])
  std=np.std(np.array(aux), axis=0)
  plt.fill_between(xx_plot, xx-std, xx+std, alpha=.3, linewidth=0, color = colours[i])
plt.show()


In [ ]:
from sklearn.manifold import TSNE, Isomap, MDS
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import manhattan_distances

#We are just including some technics for high dimensional data visualization, so you can try to see how the data is structured.
#Check https://scikit-learn.org/stable/modules/manifold.html

fig, axs = plt.subplots(2, 2,figsize=(10, 10))
fig.suptitle('Manifold Learning Examples')

# PCA
pca=PCA(n_components=2)
X_pca = pca.fit_transform(X)
axs[0,0].scatter(X_pca[y_pred!=-1][:, 0], X_pca[y_pred!=-1][:,1], marker='.', c = y_pred[y_pred!=-1], cmap=cm_preference)
axs[0,0].scatter(X_pca[y_pred==-1][:, 0], X_pca[y_pred==-1][:,1], marker='x', c = 'k')
axs[0,0].set_xlabel('PCA 1 ('+"{:.2f}".format(pca.explained_variance_ratio_[0]*100.0)+'%)')
axs[0,0].set_ylabel('PCA 2 ('+"{:.2f}".format(pca.explained_variance_ratio_[1]*100.0)+'%)')
axs[0,0].set_title("PCA Projection")


# TSNE
X_tsne = TSNE(n_components=2, perplexity=10,early_exaggeration=50,metric="l1").fit_transform(X)
axs[0,1].scatter(X_tsne[y_pred!=-1][:, 0], X_tsne[y_pred!=-1][:,1], marker='.', c = y_pred[y_pred!=-1], cmap=cm_preference)
axs[0,1].scatter(X_tsne[y_pred==-1][:, 0], X_tsne[y_pred==-1][:,1], marker='x', c = 'k')
axs[0,1].set_title("t-SNE Projection")

# Isomap
X_isomap = Isomap(n_components=2,n_neighbors=10,metric="l1").fit_transform(X)
axs[1,0].scatter(X_isomap[y_pred!=-1][:, 0], X_isomap[y_pred!=-1][:,1], marker='.', c = y_pred[y_pred!=-1], cmap=cm_preference)
axs[1,0].scatter(X_isomap[y_pred==-1][:, 0], X_isomap[y_pred==-1][:,1], marker='x', c = 'k')
axs[1,0].set_title("Isomap Projection")

# Multidimensional scaling
M=manhattan_distances(X)
X_mds = MDS(n_components=2,dissimilarity='precomputed').fit_transform(M)
axs[1,1].scatter(X_mds[y_pred!=-1][:, 0], X_mds[y_pred!=-1][:,1], marker='.', c = y_pred[y_pred!=-1], cmap=cm_preference)
axs[1,1].scatter(X_mds[y_pred==-1][:, 0], X_mds[y_pred==-1][:,1], marker='x', c = 'k')
axs[1,1].set_title("Multidimensional scaling projection")

fig.show()


In [ ]:
xmin=1.5; xmax=5.0
direct0=pca.components_[0]
direct1=pca.components_[1]
plt.figure(figsize=(14, 6))
plt.xlabel("Bound Length ($\AA$)")
plt.ylabel("PCA component direction")
xx_plot=(np.array(range(N))/float(N))*(xmax-xmin)+xmin
plt.plot(xx_plot, direct0,'-', linewidth=5, c = 'green',label='First component direction')
plt.plot(xx_plot, direct1,'-', linewidth=5, c = 'magenta',label='Second component direction')
for i in [10, 990]:
  xx=X[i]
  N=len(xx)
  xx_plot=(np.array(range(N))/float(N))*(xmax-xmin)+xmin
  plt.plot(xx_plot, xx,'--', linewidth=1, c ='k')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

df = pd.read_csv('/content/Notebooks/Oleron Solidification School 2024/data/MD_melting/lammps_thermostat_log.csv',delim_whitespace=True)
temps = df['Temp'].to_numpy()[1:]
cm_preference='jet'

# Plot again using the temperatures as colors
fig, axs = plt.subplots(2, 2,figsize=(12, 10))
fig.suptitle('Manifold Learning Examples')

# PCA
axs[0,0].scatter(X_pca[:, 0], X_pca[:,1], marker='.', c = temps, cmap=cm_preference)
axs[0,0].set_title("PCA Projection")

# TSNE
axs[0,1].scatter(X_tsne[:, 0], X_tsne[:,1], marker='.', c = temps, cmap=cm_preference)
axs[0,1].set_title("t-SNE Projection")

# Isomap
axs[1,0].scatter(X_isomap[:, 0], X_isomap[:,1], marker='.', c = temps, cmap=cm_preference)
axs[1,0].set_title("Isomap Projection")

# Multidimensional scaling
aux=axs[1,1].scatter(X_mds[:, 0], X_mds[:,1], marker='.', c = temps, cmap=cm_preference)
axs[1,1].set_title("Multidimensional scaling projection")

cax,kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
plt.colorbar(aux, cax=cax, label='Temperature (K)', **kw)

fig.show()


In [ ]:
import matplotlib.pyplot as plt
from numpy import convolve

plt.figure(figsize=(15, 5))
#N=5
#plt.plot(convolve(temps[y_pred!=-1],np.ones(N)/N)[N:-N], convolve(y_pred[y_pred!=-1],np.ones(N)/N)[N:-N], c = (0.5,0.5,0.5))
plt.scatter(temps[y_pred!=-1], y_pred[y_pred!=-1], marker='.', c = y_pred[y_pred!=-1], cmap='bwr')
plt.scatter(temps[y_pred==-1], y_pred[y_pred==-1], marker='x', c = 'k')
plt.ylim(-1.5, 1.5)
plt.yticks([-1, 0, 1])
plt.axvline(1358, color='k', linestyle='--')
plt.text(1358, 0, 'Exp. Melting Point (1358K)', ha='center', va='center',rotation='vertical', backgroundcolor=(1, 1, 1, 0.8))
plt.xlabel('Temperature')
plt.ylabel('Cluster Label')
plt.title('Temperature vs. Cluster Label')
plt.show()




-------------------------------------------------------------------------
# Clustering Local Descriptors from Al Molecular dynamics for Grain Growth

This will be very similar to the exemple above, but now we are interested in data from the paper *Atom-centered symmetry functions for constructing high-dimensional neural network potentials* ([DOI:10.1063/1.3553717](https://doi.org/10.1063/1.3553717)) provided by Noel Jakse. We will again use a `dscribe` descriptor, now LMBTR (analogous to a $g(r)$ around a particular atom) to try to pinpoint the grain formation. The data we are using is from a particular frame, and includes randomly selected atoms (script provided in the github). We will visualize a classifier for the clustering istructure we discover on fine slices of the MD, to see the nucleation and grain formation.

In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.manifold import TSNE, locally_linear_embedding
from sklearn.decomposition import PCA
import numpy as np

X = np.load('/content/Notebooks/Oleron Solidification School 2024/data/Al_nucleation_MD/lmbtr_175ps.npy')


In [ ]:
from matplotlib import cm,colormaps

N=len(X)
print(N)
cmap = colormaps['jet']
colours=[cmap(float(i)/float(N-1)) for i in range(N)]

xmin=2.0; xmax=4.5

plt.figure(figsize=(14, 6))
plt.xlabel("Bound Length ($\AA$)")
plt.ylabel("Cluster-averaged MBTR for k=2 (Radial Distribution)")
for i in range(1,len(X)):
  xx=X[i]#+0.001*i
  N=len(xx)
  xx_plot=(np.array(range(N))/float(N))*(xmax-xmin)+xmin
  plt.plot(xx_plot, xx,'-', linewidth=1, c = colours[i])


plt.show()

#How this data compare to the case above? Why they are so different?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Lets have a look on how diferent our plots are between thenselves...
distances = []
distances_small = []
for i in range(len(X)):
  for j in range(i+1, len(X)):
    aux=np.linalg.norm(X[i] - X[j], ord=1)
    if aux<2:
      distances_small.append(aux)
    distances.append(aux)

plt.hist(distances, bins=100)
plt.xlabel('L1 Distance')
plt.ylabel('Frequency')
plt.title('Histogram of L1 Distances Between Elements of X')
plt.show()

#Lets zoom on the lowest distances...
plt.hist(distances_small, bins=200)
plt.xlabel('L1 Distance')
plt.ylabel('Frequency')
plt.title('Histogram of L1 Distances Between Elements of X')
plt.show()


In [ ]:
from sklearn.cluster import KMeans, DBSCAN
import matplotlib

cm_preference='jet'
nb_c = 5
model = KMeans(n_clusters=nb_c
               , init='random'
               , n_init=100
               , max_iter=1000
               , tol=1e-04
              )
# For fun, you can check this one also! What happens? Can you explain?
# OBS.: Remember to rerun this cell with only the kmeans before continuing!
#model = DBSCAN(eps=1.5,
#               min_samples=5,
#               metric='l1')

model.fit(X)

y_pred = model.labels_
nb_c=max(y_pred)+1

print(nb_c)

cmap = matplotlib.cm.get_cmap(cm_preference)
colours=[cmap(float(i)/float(nb_c-1)) for i in range(nb_c)]

xmin=2.0; xmax=4.5

plt.figure(figsize=(14, 6))
plt.xlabel("Bound Length ($\AA$)")
plt.ylabel("Cluster-averaged MBTR for k=2 (Radial Distribution)")
for i in range(nb_c):
  aux=X[y_pred==i]
  xx=np.mean(np.array(aux), axis=0)
  N=len(xx)
  xx_plot=(np.array(range(N))/float(N))*(xmax-xmin)+xmin
  plt.plot(xx_plot, xx,'-', linewidth=5, c = colours[i])
  std=np.std(np.array(aux), axis=0)
  plt.fill_between(xx_plot, xx-std, xx+std, alpha=.3, linewidth=0, color = colours[i])
plt.show()


In [ ]:
#Vizualizing the data structure...

from sklearn.manifold import TSNE, Isomap, MDS
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import manhattan_distances

X_pca = PCA(n_components=2).fit_transform(X)
X_tsne = TSNE(n_components=2, perplexity=30,early_exaggeration=1.5,metric="l1").fit_transform(X)
X_isomap = Isomap(n_components=2,n_neighbors=30,metric="l1").fit_transform(X)
M=manhattan_distances(X)
X_mds = MDS(n_components=2,dissimilarity='precomputed').fit_transform(M)

In [ ]:
fig, axs = plt.subplots(2, 2,figsize=(10, 10))
fig.suptitle('Manifold Learning Examples')

# PCA
axs[0,0].scatter(X_pca[y_pred!=-1][:, 0], X_pca[y_pred!=-1][:,1], marker='.', c = y_pred[y_pred!=-1], cmap=cm_preference)
axs[0,0].scatter(X_pca[y_pred==-1][:, 0], X_pca[y_pred==-1][:,1], marker='x', c = 'k',alpha=0.1)
axs[0,0].set_title("PCA Projection")


# TSNE
axs[0,1].scatter(X_tsne[y_pred!=-1][:, 0], X_tsne[y_pred!=-1][:,1], marker='.', c = y_pred[y_pred!=-1], cmap=cm_preference)
axs[0,1].scatter(X_tsne[y_pred==-1][:, 0], X_tsne[y_pred==-1][:,1], marker='x', c = 'k',alpha=0.1)
axs[0,1].set_title("t-SNE Projection")

# Isomap
axs[1,0].scatter(X_isomap[y_pred!=-1][:, 0], X_isomap[y_pred!=-1][:,1], marker='.', c = y_pred[y_pred!=-1], cmap=cm_preference)
axs[1,0].scatter(X_isomap[y_pred==-1][:, 0], X_isomap[y_pred==-1][:,1], marker='x', c = 'k',alpha=0.1)
axs[1,0].set_title("Isomap Projection")

# Multidimensional scaling
axs[1,1].scatter(X_mds[y_pred!=-1][:, 0], X_mds[y_pred!=-1][:,1], marker='.', c = y_pred[y_pred!=-1], cmap=cm_preference)
axs[1,1].scatter(X_mds[y_pred==-1][:, 0], X_mds[y_pred==-1][:,1], marker='x', c = 'k',alpha=0.1)
axs[1,1].set_title("Multidimensional scaling projection")

fig.show()


In [ ]:
#Lets now use this cluster labels to train a classifier!

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

mlogreg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
Xm_train, Xm_test, ym_train, ym_test = train_test_split(X, y_pred, test_size=0.80, random_state=123)

mlogreg.fit(Xm_train, ym_train)
ym_pred = mlogreg.predict(X)
proba_log = mlogreg.predict_proba(X)
prob_pred=[proba_log[i,ym_pred[i]] for i in range(len(ym_pred))]


If you dont know what is a Confussion Matrix, check: https://fr.wikipedia.org/wiki/Matrice_de_confusion

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


# Compute confusion matrix
cm = confusion_matrix(y_pred, ym_pred)

# Visualize confusion matrix
plt.figure()
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Clustering Labels")
plt.ylabel("Model Labels")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
#Lets now see how those classes are spacialy distributed in the molecular dynamics frames!
gdrs=[]
poss=[]
times=[166,170,174,175,177]
for t in times:
  gdrs.append(np.load('/content/Notebooks/Oleron Solidification School 2024/data/Al_nucleation_MD/lmbtr_'+str(t)+'ps.pos.npy'))
  poss.append(np.load('/content/Notebooks/Oleron Solidification School 2024/data/Al_nucleation_MD/slice_'+str(t)+'ps.pos.npy'))


fig, axs = plt.subplots(1,5,figsize=(22,4))
fig.suptitle('Spacial distribution of labels')

for i in range(5):
  gdr=gdrs[i]
  pos=poss[i]
  colours= mlogreg.predict(gdr)
  axs[i].scatter(pos[:,0],pos[:,1],marker='.', c = colours, cmap=cm_preference)
  axs[i].set_title("t="+str(times[i])+"ps")
  axs[i].set_xlabel('Position X ($\t{\AA}$ )')

axs[0].set_ylabel('Position Y ($\t{\AA}$ )')

fig.show()

# Can you see when the grain apear? Can we count the grain size with it?
# What you think the model learned about the pomains? And about the liquid?

------------------------------------------
# Checking the chalengy of organizing Local Minima structures with Cu$_{147}$ nanoclusters

Here, we are checking some MBTR or Coulomb Matrixes that were computed for a local minima structures colection we generated with help of `dscribe` and [ABCluster](https://doi.org/10.1039/C5CP04060D). This is here more to showcase the complicated space we have where most of the structures are in a high energy configuration and only a few are realy closer to eachother and to the global minima configuration. This is didatical, because its very close to many real cenarious. We invite you to play with the data and try to propose analysis.

In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
import numpy as np

y = np.load('/content/Notebooks/Oleron Solidification School 2024/data/Cu147_nanoclusters/energies.npy')
#X = np.load('/content/Notebooks/Oleron Solidification School 2024/data/Cu147_nanoclusters/mbtr.npy')
X = np.load('/content/Notebooks/Oleron Solidification School 2024/data/Cu147_nanoclusters/cm.npy')
X=X[y<-454]
y=y[y<-454]

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)

# Percentage of variance explained for each components
print(
    "explained variance ratio (first two components): %s"
    % str(pca.explained_variance_ratio_)
)

plt.figure()
dist=plt.scatter(X_r[:,0], X_r[:,1], marker='.', c=y, cmap='jet')
plt.title("PCA of the Chosen Representation")
cbar = plt.colorbar(dist)
cbar.set_label('Energy', rotation=90)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.manifold import TSNE, locally_linear_embedding
import numpy as np

##LOADING FILES
#X = np.load('/content/Notebooks/Oleron Solidification School 2024/data/Cu147_nanoclusters/mbtr.npy')
X = np.load('/content/Notebooks/Oleron Solidification School 2024/data/Cu147_nanoclusters/cm.npy')
y = np.load('/content/Notebooks/Oleron Solidification School 2024/data/Cu147_nanoclusters/energies.npy')

#Checking the energy distribution
fig, ax = plt.subplots()
ax.set_title("Energy Histogram")
ax.set_xlabel("Potential Energy (eV)")
ax.set_ylabel("Number of Structures")
N, bins, patches = ax.hist(y, bins=80, color="b")
plt.show()


#Lets try to visualize some of the data....
X=X[y<-453]
y=y[y<-453]
#X_r, _ = locally_linear_embedding(X,n_neighbors=50, n_components=2)
X_r = TSNE(n_components=2,perplexity=5,early_exaggeration=10).fit_transform(X)

plt.figure()
dist=plt.scatter(X_r[:,0], X_r[:,1], marker='.', c=y, cmap='jet')
plt.title("PCA of the Chosen Representation")
cbar = plt.colorbar(dist)
cbar.set_label('Energy', rotation=90)
plt.show()

In [ ]:

plt.figure(figsize=(12,10))
dist=plt.scatter(X_r[:,0], X_r[:,1], marker='o', c=y, cmap='jet')
plt.title("PCA of the Chosen Representation")
cbar = plt.colorbar(dist)
cbar.set_label('Potential Energy (eV)', rotation=90)
plt.show()


-------------------------------------------
# Challenge: Learning the Potential of Morse Pairs from LMBTR Bulk Data

Lets re-take the data from the MD we considered in the fist section. Instead of lookint to global descriptors, we are now loofing into local ones for 1000 diferent atoms, each one from one diferent frame of our MD. Additionaly, the atomic contribution for the total potential energy is also done.

We will use this data to train a potential energy model using a ANN, similar to the one in the regression notebook. We will test this in some artificialy generated data for a dimer of copper atoms, for wich we know the ground trouth of the interaction energy (the Morse Potential). Try to play a little bit with the NN structure and training. The initial implementation we are giving here is ok, but can be inpruved. How can it be done?

In [ ]:
import numpy as np

X = np.load('/content/Notebooks/Oleron Solidification School 2024/data/MD_melting/lmbtr_Cuatoms_i_i.npy')
y = np.load('/content/Notebooks/Oleron Solidification School 2024/data/MD_melting/pe_Cuatoms_i_i.npy')

#X = np.load('/content/lmbtr_Cuatoms_dimer.npy')
#y = np.load('/content/pe_Cuatoms_dimer.npy')

print(X.shape)
print(y.shape)

RNG_SEED = 171
np.random.seed(RNG_SEED)
N_sample = len(X)
N_train = int(0.70 * N_sample)
N_test = N_sample

ind_train = np.random.choice(range(N_sample), size = N_train, replace = False) #create the training set
ind_test = np.array(list(i for i in range(N_sample) if i not in ind_train)) #create the testset with remaining elements not in ind list

x_data = X
y_data = y

#pick up points in the indices' array
x_train = x_data[np.array(ind_train)]
y_train = y_data[np.array(ind_train)]
print('train set size: ', y_train.shape)

#pick up points in the indices' array not in the train (very important to avoid bias in the testing
x_test = x_data[np.array(ind_test)]
y_test = y_data[np.array(ind_test)]
print('test set size: ', y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# Define the ANN model/archtecture
# Question: How many parameters this function has? We have 1000 data points...
NN=10
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(100, NN)
        self.fc2 = nn.Linear(NN, NN)
        self.fc3 = nn.Linear(NN, NN)
        self.fc4 = nn.Linear(NN, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.fc4(x)
        return x

# Create the ANN model
ann = ANN()

# Define the loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(ann.parameters(), lr=0.001)

# Convert the data to PyTorch tensors
x_data = torch.tensor(x_data, dtype=torch.float32)
y_data = torch.tensor(y_data, dtype=torch.float32)
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


val_plot=[]
loss_plot=[]
# Train the ANN model
for epoch in range(5000):
    # Forward pass
    y_pred = ann(x_train)
    y_val = ann(x_test)

    # Compute the loss
    loss = loss_fn(y_pred, y_train)
    val = loss_fn(y_val, y_test)
    val_plot.append(val.item())
    loss_plot.append(loss.item())

    # Print the loss every 100 epochs
    if (epoch+1) % 100 == 0:
      print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Val: {loss.item():.4f}')


    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


################################################
#
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title("Multi-layer Perceptron")
ax.set_xlabel("Iterations")
ax.set_ylabel("Loss")
ax.set_xscale('log')
ax.set_yscale('log')

ax.plot(loss_plot,label="Train set")
ax.plot(val_plot,label="Validation set")
ax.legend()
plt.show()

In [ ]:
from ase import Atoms
from dscribe.descriptors import LMBTR

# Create a series of ase atoms objects containing two Cu atoms
# Question: Is this data in the same class of the training? We expect our model to perform good here?
atoms = []
N=100
dist=[1.5+i*(5.0-1.5)/float(N) for i in range(N)]
for d in dist:
    dimmer = Atoms('Cu2', positions=[(0, 0, 0), (0, 0, d)])
    atoms.append(dimmer)

# Use dscribe to compute the LMBTR for the first atom in each object
lmbtr = LMBTR(
    species=["Cu"],
    geometry={"function": "distance"},
    grid={"min": 1.5, "max": 5, "n": 100, "sigma": 0.05},
    weighting={"function": "exp", "scale": 0.5, "threshold": 1e-3},
    periodic=True,
    normalization="l2")

discripts=[]
for dimmer in atoms:
    disc=lmbtr.create(dimmer, centers=[0])
    discripts.append(disc[0][100:200])

# Convert the sample data to a PyTorch tensor
x_sample = torch.tensor(discripts, dtype=torch.float32)

# Predict the output for the sample data
y_pred_samples = ann(x_sample).detach().numpy()
y_pred_samples=[y_pred_samples[i]-y_pred_samples[86] for i in range(len(y_pred_samples))] #apply rcut in 4.5 (dist[86])


# Computing real value:
D = 0.3429/23.0;r = 2.866; a = 1.3588; rcut=4.5
delta=2.0*D*(np.exp(-2*a*(rcut-r))-2*np.exp(-a*(rcut-r)))
V_real = [2.0*D*(np.exp(-2*a*(d-r))-2*np.exp(-a*(d-r)))-delta  for d in dist]

plt.plot(dist,y_pred_samples,'o',label="Predicted")
plt.plot(dist,V_real,'o',label="Real")
plt.legend()
plt.show()